### Dataframe style looks better

In [12]:
%load_ext autoreload
%autoreload 2
%pylab inline
import os
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [2]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/speech_commands' # to concatenate audios
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Command_Dataframe.pkl')

In [3]:
all_spks = list(data_df.spk.unique())
all_words = list(data_df.sent.unique())

## Manifest For Trial

common word를 정한다.  
common word는 등록된 command set이고,  (i.e., "ok", "google")  
나머지 unknown words는 등록되지 않은 set이라고 생각하면 된다. (따라서 si_train에 보이지 않는다.) (i.e., "seoul", "incheon")  

In [4]:
n_common_words = 20 # <= 20
common_words = np.random.choice(data_df.sent.value_counts().index[:20], 20, replace=False)
common_words

array(['no', 'nine', 'off', 'one', 'seven', 'up', 'yes', 'eight', 'four',
       'three', 'right', 'left', 'two', 'five', 'down', 'zero', 'on',
       'stop', 'six', 'go'], dtype=object)

In [5]:
unknown_words = list(set(all_words) - set(common_words))
unknown_words

['house',
 'wow',
 'sheila',
 'cat',
 'happy',
 'tree',
 'dog',
 'marvin',
 'bird',
 'bed']

In [6]:
common_words_df = data_df[data_df.sent.isin(common_words)]
unknown_words_df = data_df[data_df.sent.isin(unknown_words)]

### Enroll DataFrame

In [7]:
enroll_candidate = common_words_df.spk.value_counts().index

각 common words를 4개 이상 가진 스피커를 enroll speaker로 둔다.

In [8]:
enroll_spks = []
for spk in enroll_candidate:
    word_counts = common_words_df[common_words_df.spk == spk].groupby(['sent']).size()
    if np.count_nonzero(word_counts > 3) == len(common_words):
        enroll_spks.append(spk)

In [9]:
print("number of enroll spks: {}".format(len(enroll_spks)))

number of enroll spks: 146


각 word의 몇 개의 발화를 enroll에 쓸건지 정한다.

In [10]:
n_enroll_uttrs = 2
positive_df = common_words_df[common_words_df.spk.isin(enroll_spks)] # utterances from enroll spks
grouped = positive_df.groupby(['spk', 'sent'], as_index=False, group_keys=False)
enroll_df = grouped.apply(lambda x: x.sample(n=n_enroll_uttrs)) # enroll utterances for each word

trial이 저장될 폴더

In [13]:
dir_name = "trials/commands/{}word_{}uttrs".format(len(common_words), n_enroll_uttrs)
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)

In [14]:
enroll_df.to_pickle("{}/enroll.pkl".format(dir_name))

### EnrollSpeaker's Test Utterance DataFrame

In [15]:
enroll_idx = [idx[-1] for idx in enroll_df.index]
pos_test_df = data_df[data_df.spk.isin(enroll_spks)].drop(index=enroll_df.index) # test_uttrs contain unknown words

In [16]:
pos_test_df.to_pickle("{}/posTest.pkl".format(dir_name))

### Imposter's Test Utterance DataFrame

In [17]:
not_enroll_df = data_df[~data_df.spk.isin(enroll_spks)]

imposter_df는 오히려 uttrs를 적게 가지고 있는 spk들을 중심으로 뽑았다.  
그리고 word는 common이랑 unknown을 모두 가지고 있다.

In [18]:
imposter_spks = not_enroll_df.spk.value_counts().index[-300:]
imposter_df = not_enroll_df[not_enroll_df.spk.isin(imposter_spks)]

In [19]:
imposter_df.to_pickle("{}/negTest.pkl".format(dir_name))

#### For KWS or SI DataFrame

Imposter와 겹치면 안된다

In [20]:
kws_df = not_enroll_df.drop(index=imposter_df.index)

In [21]:
kws_df.to_pickle("{}/kws.pkl".format(dir_name))

#### Check

In [22]:
(set(enroll_df.index) & set(pos_test_df.index))

set()

In [23]:
(set(imposter_df.index) & set(pos_test_df.index))

set()

In [24]:
(set(imposter_df.index) | set(pos_test_df.index) | set(enroll_df.index)) & (set(kws_df.index))

set()

## Audio Concatenate

enroll을 제외한 (pos|neg) test utterances만 concat하면 된다.  
e.g. "yes" + "one" + "left" --> "yes one left"  
차라리 vad를 써서 중간에 공백을 적당히 넣는게 낫지않을까?

In [25]:
n_words_in_uttr = 3
grouped = pos_test_df.groupby(['spk'], as_index=True, group_keys=False)

In [26]:
uttrs_per_spk = grouped 